<a href="https://colab.research.google.com/github/tirabo/Algoritmos-y-Programacion/blob/main/objetos2_2021_05_26_Algoritmos_y_Programaci%C3%B3n_tipos_de_datos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tipos abstractos de datos

La clase pasada vimos un ejemplo de tipo abstracto, o en la jerga de lenguajes orientados a objetos, definición de una clase: la clase Fecha.

Para ello utilizamos algunas de las funciones que ya desarrollamos en lecciones anteriores:


In [ ]:
def es_bisiesto(anho: int) -> bool:
    # pre: anho > 0
    # post: devuelve True si anho  es divisible por 4 (NO corresponde siempre a un año bisiesto)
    precondición = type(anho) == int and anho > 0
    assert precondición, 'Error: anho debe ser un entero positivo.'

    return anho % 4 == 0 and anho % 100 != 0 or anho % 400 == 0

def es_fecha_valida(fecha: tuple) -> bool:
    # pre: fecha es una terna de enteros positivos
    # post: devuelve True sii la terna se corresponde con el día, mes y año de una fecha válida
    precondición = type(fecha) == tuple and all(type(x) == int and x > 0 for x in fecha)
    assert precondición, 'Error: fecha debe ser una terna de enteros positivos.'

    dia, mes, anho = fecha

    anho_ok = 1 <= anho
    mes_ok = 1 <= mes <= 12
    if mes in [4, 6, 9, 11]:
        dia_ok = 1 <= dia <= 30
    elif mes == 2:
        dia_ok = 1 <= dia <= 28 or (dia == 29 and es_bisiesto(anho))
    else:
        dia_ok = 1 <= dia <= 31
    
    return dia_ok and mes_ok and anho_ok

def es_fecha_valida_str(cadena: str) -> bool:
    # pre: cadena es una cadena de caracteres
    # post: devuelve True sii la cadena contiene una fecha en formato dia/mes/año
    precondición = type(cadena) == str
    assert type(cadena) == str, 'Error: cadena debe ser una cadena de caracteres.'

    cadena_sin_espacios = ''.join(cadena.split())
    cadenas_fecha = cadena_sin_espacios.split('/')
    res = len(cadenas_fecha) == 3 and all(cadena.isdecimal() for cadena in cadenas_fecha)
    if res:
        terna = tuple(int(cad) for cad in cadenas_fecha)
        res = es_fecha_valida(terna)
    return res

def bisiestos_hasta(anho: int) -> int:
    # pre: anho es un año válido
    # post: devuelve el número de años bisiestos pasados incluyendo anho, si es bisiesto
    precondición = type(anho) == int and anho > 0
    assert precondición, 'Error: anho debe ser un entero positivo.'

    bisiestos_anteriores = anho // 4 - anho // 100 + anho // 400
    return bisiestos_anteriores

def dias_del_anho_actual(fecha: tuple) -> int:
    # pre: fecha es una fecha válida
    # post: devuelve el número de días transcurridos en el corriente año, contando el actual
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    # [días antes de enero, días antes de febrero, etc.] de un año no bisiesto
    DIAS_MESES_ANTERIORES = [0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334]
    dia, mes, anho = fecha

    dias = dia + DIAS_MESES_ANTERIORES[mes-1]
    # si el año es bisiesto y la fecha es después de febrero, pasó un día más
    if es_bisiesto(anho) and mes > 2:
        dias = dias + 1
    return dias

def dias_de_la_era(fecha: tuple) -> int:
    # pre: fecha es una fecha válida
    # post: devuelve el número de días desde el 1/1/1 (asumiendo siempre Gregoriano) hasta la fecha, contando ambos extremos
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    dia, mes, anho = fecha

    nro_de_dias = (anho - 1) * 365 + bisiestos_hasta(anho-1) + dias_del_anho_actual(fecha)
    return nro_de_dias

def dia_de_la_semana(fecha: tuple) -> str:
    # pre: fecha es una fecha válida
    # post: devuelve el día de la semana en que cae la fecha dada
    precondición = es_fecha_valida(fecha)
    assert precondición, 'Error: fecha debe ser una fecha válida.'

    DIAS_DE_LA_SEMANA = ['domingo', 'lunes', 'martes', 'miércoles', 'jueves', 'viernes', 'sábado']
    return DIAS_DE_LA_SEMANA[dias_de_la_era(fecha) % 7]

def ingresar_fecha() -> tuple:
    # post: devuelve la primera terna de enteros positivos ingresada con formato día/mes/año

    esperando_fecha = True
    while esperando_fecha:
        cadena_ingresada = input('Ingrese fecha con formato día/mes/año: ')
        cadena_sin_espacios = ''.join(cadena_ingresada.split())
        cadenas_fecha = cadena_sin_espacios.split('/')
        if len(cadenas_fecha) == 3 and all(cadena.isdecimal() for cadena in cadenas_fecha):
            terna = [int(cadena) for cadena in cadenas_fecha]
            esperando_fecha = False
        else:
            print('Error: debe ingresar una fecha con formato día/mes/año, donde día, mes y año deben ser números enteros positivos. Inténtelo nuevamente.')
    return terna

def ingresar_fecha_valida() -> tuple:
    fecha = ingresar_fecha()
    while not es_fecha_valida(fecha):
        print('Error: debe ingresar una fecha válida. Inténtelo nuevamente.')
        fecha = ingresar_fecha()
    return fecha


Gradualmente fuimos escribiendo la siguiente clase, donde cambiamos la representación interna, con ternas, por una con 3 atributos diferentes. Además, hemos elegido que sea mutable (observar por ejemplo el método `establecer_anho()`.

In [ ]:
from datetime import date

class Fecha:                        # una definición mutable
    def __init__(self, dma = None):
        assert dma == None or type(dma) == tuple and es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        if dma == None:
            hoy = date.today()
            self.__dia = hoy.day
            self.__mes = hoy.month
            self.__anho = hoy.year
        else:
            self.__dia = dma[0]
            self.__mes = dma[1]
            self.__anho = dma[2]
    def dma(self):
        return (self.__dia, self.__mes, self.__anho)
    def amd(self):
        return (self.__anho, self.__mes, self.__dia)
    def dia(self):
        return self.__dia
    def mes(self):
        return self.__mes
    def anho(self):
        return self.__anho
    def siglo(self):
        return self.__anho // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and es_fecha_valida((self.__dia, self.__mes, anho)), 'Error: intento de establecer un año no válido'
        self.__anho = anho
    def establecer_mes(self, mes):
        assert type(mes) == int and es_fecha_valida((self.__dia, mes, self.__anho)), 'Error: intento de establecer un mes no válido'
        self.__mes = mes
    def establecer_dia(self, dia):
        assert type(dia) == int and es_fecha_valida((dia, self.__mes, self.__anho)), 'Error: intento de establecer un día no válido'
        self.__dia = dia
    def cadena(self):
        return str(self.__dia) + '-' + str(self.__mes) + '-' + str(self.__anho)
    def __str__(self):
        return str(self.__dia) + '/' + str(self.__mes) + '/' + str(self.__anho)

In [ ]:
fecha = Fecha()
fecha2 = Fecha((23,5,2021))
print(fecha.dma())
print(fecha2.dma())
print(fecha)
print(fecha2)
print(fecha.cadena())

(26, 5, 2021)
(23, 5, 2021)
26/5/2021
23/5/2021
26-5-2021


Respecto a lo visto durante la última clase, la versión anterior añade:

*   Una representación mutable, independiente de listas o tuplas, con 3 campos: uno para el día, otro para el mes y otro para el año.
*   La posibilidad de no proporcionar ningún argumento, dando por hecho que refiere al día de hoy
*   La definición del método `__str__` que hace posible una utilización interesante de print.

¿Cómo agregar la posibilidad de leer una fecha a partir de una cadena de caracteres?

In [ ]:
from datetime import date

class Fecha:                        # una definición mutable
    def __init__(self, dma = None):
        assert dma == None or type(dma) == str and es_fecha_valida_str(dma) or type(dma) == tuple and es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        if dma == None:
            hoy = date.today()
            self.__dia = hoy.day
            self.__mes = hoy.month
            self.__anho = hoy.year
        elif type(dma) == str:
            cadena_sin_espacios = ''.join(dma.split())
            cadenas_fecha = cadena_sin_espacios.split('/')
            terna = [int(cad) for cad in cadenas_fecha]
            self.__dia = terna[0]
            self.__mes = terna[1]
            self.__anho = terna[2]
        else:
            self.__dia = dma[0]
            self.__mes = dma[1]
            self.__anho = dma[2]
    def dma(self):
        return (self.__dia, self.__mes, self.__anho)
    def amd(self):
        return (self.__anho, self.__mes, self.__dia)
    def dia(self):
        return self.__dia
    def mes(self):
        return self.__mes
    def anho(self):
        return self.__anho
    def siglo(self):
        return self.__anho // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and es_fecha_valida((self.__dia, self.__mes, anho)), 'Error: intento de establecer un año no válido'
        self.__anho = anho
    def establecer_mes(self, mes):
        assert type(mes) == int and es_fecha_valida((self.__dia, mes, self.__anho)), 'Error: intento de establecer un mes no válido'
        self.__mes = mes
    def establecer_dia(self, dia):
        assert type(dia) == int and es_fecha_valida((dia, self.__mes, self.__anho)), 'Error: intento de establecer un día no válido'
        self.__dia = dia
    def __str__(self):
        return str(self.__dia) + '/' + str(self.__mes) + '/' + str(self.__anho)

In [ ]:
fecha = Fecha()
fecha2 = Fecha((23,5,2021))
print(fecha)
print(fecha2)
fecha3 = Fecha('26/5/2021')
fecha4 = Fecha('26/5/2021')
fecha5 = fecha3
print(fecha3)
print(fecha4)
print(fecha5)
print(fecha4 == fecha3)
print(fecha5 == fecha3)

26/5/2021
23/5/2021
26/5/2021
26/5/2021
26/5/2021
False
True


Por suerte la igualdad se puede redefinir:

In [ ]:
from datetime import date

class Fecha:                        # una definición mutable
    def __init__(self, dma = None):
        assert dma == None or type(dma) == str and es_fecha_valida_str(dma) or type(dma) == tuple and es_fecha_valida(dma), 'Error: intento de crear una fecha no válida.'
        if dma == None:
            hoy = date.today()
            self.__dia = hoy.day
            self.__mes = hoy.month
            self.__anho = hoy.year
        elif type(dma) == str:
            cadena_sin_espacios = ''.join(dma.split())
            cadenas_fecha = cadena_sin_espacios.split('/')
            terna = [int(cad) for cad in cadenas_fecha]
            self.__dia = terna[0]
            self.__mes = terna[1]
            self.__anho = terna[2]
        else:
            self.__dia = dma[0]
            self.__mes = dma[1]
            self.__anho = dma[2]
    def dma(self):
        return (self.__dia, self.__mes, self.__anho)
    def amd(self):
        return (self.__anho, self.__mes, self.__dia)
    def dia(self):
        return self.__dia
    def mes(self):
        return self.__mes
    def anho(self):
        return self.__anho
    def siglo(self):
        return self.__anho // 100 + 1
    def establecer_anho(self, anho):
        assert type(anho) == int and es_fecha_valida((self.__dia, self.__mes, anho)), 'Error: intento de establecer un año no válido'
        self.__anho = anho
    def establecer_mes(self, mes):
        assert type(mes) == int and es_fecha_valida((self.__dia, mes, self.__anho)), 'Error: intento de establecer un mes no válido'
        self.__mes = mes
    def establecer_dia(self, dia):
        assert type(dia) == int and es_fecha_valida((dia, self.__mes, self.__anho)), 'Error: intento de establecer un día no válido'
        self.__dia = dia
    def __str__(self):
        return str(self.__dia) + '/' + str(self.__mes) + '/' + str(self.__anho)
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Fecha), 'Error: otro debe ser instancia de Fecha.'
        return self.__anho == otro.__anho and self.__mes == otro.__mes and self.__dia == otro.__dia
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Fecha), 'Error: otro debe ser instancia de Fecha.'
        anho_menor = self.__anho < otro.__anho
        mes_menor = self.__anho == otro.__anho and self.__mes < otro.__mes
        dia_menor = self.__anho == otro.__anho and self.__mes == otro.__mes and self.__dia < otro.__dia
        return anho_menor or mes_menor or dia_menor
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Fecha), 'Error: otro debe ser instancia de Fecha.'
        return self < otro or self == otro


In [ ]:
fecha = Fecha()
fecha2 = Fecha((23,5,2021))
print(fecha)
print(fecha2)
fecha3 = Fecha('26/5/2021')
fecha4 = Fecha('26/5/2021')
fecha5 = fecha3
print(fecha3)
print(fecha4)
print(fecha5)
print(fecha4 != fecha3)
print(fecha5 == fecha3)
print(fecha3 > fecha4)

26/5/2021
23/5/2021
26/5/2021
26/5/2021
26/5/2021
False
True
False


Un caso similar es para el cómputo del tiempo con horas minutos y segundos:

In [ ]:
class Tiempo:
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        segundos = dhms[3]
        self.__segundos = segundos % 60
        minutos = segundos // 60 + dhms[2]
        self.__minutos = minutos % 60
        horas = minutos // 60 + dhms[1]
        self.__horas = horas % 24
        dias = horas // 24 + dhms[0]
        self.__dias = dias
    # getters
    def dhms(self):
        return (self.__dias, self.__horas, self.__minutos, self.__segundos)
    def dias(self):
        return self.__dias
    def horas(self):
        return self.__horas
    def minutos(self):
        return self.__minutos
    def segundos(self):
        return self.__segundos
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__dias = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__horas = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: horas debe ser un número entero no negativo menor que 60'
        self.__minutos = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: horas debe ser un número entero no negativo menor que 60'
        self.__segundos = segundos
    def __str__(self):
        return str(self.__dias) + 'd' + str(self.__horas) + 'h' + str(self.__minutos) + 'm' + str(self.__segundos) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self.__dias == otro.__dias and self.__horas == otro.__horas and self.__minutos == otro.__minutos and self.__segundos == otro.__segundos
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        dias_menor = self.__dias < otro.__dias
        horas_menor = self.__dias == otro.__dias and self.__horas < otro.__horas
        minutos_menor = self.__dias == otro.__dias and self.__horas == otro.__horas and self.__minutos < otro.__minutos
        segundos_menor = self.__dias == otro.__dias and self.__horas == otro.__horas and self.__minutos == otro.__minutos and self.__segundos < otro.__segundos
        return dias_menor or horas_menor or minutos_menor or segundos_menor
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro


In [ ]:
print(Tiempo())
tiempo = Tiempo((10,13,32,18))
print(tiempo)
print(tiempo > Tiempo())
tiempo.set_dias(0)
print(tiempo)
tiempo.set_horas(0)
print(tiempo)
tiempo.minutos = 0
print(tiempo)
tiempo.set_segundos(0)
print(tiempo)
print(tiempo == Tiempo())

0d0h0m0s
10d13h32m18s
True
0d13h32m18s
0d0h32m18s
0d0h32m18s
0d0h32m0s
False


Queremos agregar la suma de tiempos. Antes de eso resulta conveniente cambiar la representación, utilizando un arreglo en lugar de varios campos.

In [ ]:
class Tiempo:
    __DIAS = 0
    __HORAS = 1
    __MINUTOS = 2
    __SEGUNDOS = 3
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        segundos = dhms[3]
        minutos = segundos // 60 + dhms[2]
        segundos = segundos % 60
        horas = minutos // 60 + dhms[1]
        minutos = minutos % 60
        dias = horas // 24 + dhms[0]
        horas = horas % 24
        self.__tiempo = [dias, horas, minutos, segundos]
    # getters
    def dhms(self):
        return tuple(self.__tiempo)
    def dias(self):
        return self.__tiempo[Tiempo.__DIAS]
    def horas(self):
        return self.__tiempo[Tiempo.__HORAS]
    def minutos(self):
        return self.__tiempo[Tiempo.__MINUTOS]
    def segundos(self):
        return self.__tiempo[Tiempo.__SEGUNDOS]
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__tiempo[Tiempo.__DIAS] = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__tiempo[Tiempo.__HORAS] = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: minutos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__MINUTOS] = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: segundos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__SEGUNDOS] = segundos
    def __str__(self):
        return str(self.dias()) + 'd' + str(self.horas()) + 'h' + str(self.minutos()) + 'm' + str(self.segundos()) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i == len(self.__tiempo)
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i < len(self.__tiempo) and self.__tiempo[i] < otro.__tiempo[i]
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro


In [ ]:
print(Tiempo())
tiempo = Tiempo((10,13,32,18))
print(tiempo)
print(tiempo > Tiempo())
tiempo.set_dias(0)
print(tiempo)
tiempo.set_horas(0)
tiempo.set_minutos(0)
tiempo.set_segundos(0)
print(tiempo == Tiempo())

0d0h0m0s
10d13h32m18s
True
0d13h32m18s
True


Recuerden que las constantes, en Python, son simplemente variables que decidimos no modificar. Al iniciar su nombre con "__" logramos "esconderlas", es decir, que no puedan accederse ni modificarse desde afuera.

¿Qué diferencia hay con variables como `__tiempo`, que hemos llamado "campos"? Al definirlas fuera de las funciones, se convierten en variables o constantes de la clase, no de los objetos. Es decir, que esas variables o constantes son compartidas por todas las instancias de la clase.

En este caso se trata de constantes, entonces no nos molesta que sean compartidas por la clase ya que nunca se modificarán.

Pero miren lo que podría ocurrir si fueran variables. Agreguemos un método para cambiar una de esas "constantes":

In [ ]:
class Tiempo:
    __DIAS = 0
    __HORAS = 1
    __MINUTOS = 2
    __SEGUNDOS = 3
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        segundos = dhms[3]
        minutos = segundos // 60 + dhms[2]
        segundos = segundos % 60
        horas = minutos // 60 + dhms[1]
        minutos = minutos % 60
        dias = horas // 24 + dhms[0]
        horas = horas % 24
        self.__tiempo = [dias, horas, minutos, segundos]
    def cambiar_constante(self): # ESTE MÉTODO ES SOLAMENTE PARA COMPROBAR QUE __DIAS ES UNA SOLA PARA TODAS LAS INSTANCIAS DE LA CLASE
        Tiempo.__DIAS = 3
    # getters
    def dhms(self):
        return tuple(self.__tiempo)
    def dias(self):
        return self.__tiempo[Tiempo.__DIAS]
    def horas(self):
        return self.__tiempo[Tiempo.__HORAS]
    def minutos(self):
        return self.__tiempo[Tiempo.__MINUTOS]
    def segundos(self):
        return self.__tiempo[Tiempo.__SEGUNDOS]
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__tiempo[Tiempo.__DIAS] = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__tiempo[Tiempo.__HORAS] = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: minutos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__MINUTOS] = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: segundos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__SEGUNDOS] = segundos
    def __str__(self):
        return str(self.dias()) + 'd' + str(self.horas()) + 'h' + str(self.minutos()) + 'm' + str(self.segundos()) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i == len(self.__tiempo)
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i < len(self.__tiempo) and self.__tiempo[i] < otro.__tiempo[i]
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro


In [ ]:
tiempo1 = Tiempo((10,13,32,18))
tiempo2 = Tiempo((1,3,56,48))
print(tiempo1)
print(tiempo2)
print(tiempo1 > tiempo2)
print(tiempo1 == Tiempo())
tiempo1.cambiar_constante()
print(tiempo1)
print(tiempo2)

10d13h32m18s
1d3h56m48s
True
False
18d13h32m18s
48d3h56m48s


Como dijimos, dado que en nuestro caso son constantes, no hay problema alguno en definirlos para toda la clase.

¿Puede tener sentido tener variables de la clase (además de las variables de las instancias)? Puede ocurrir que nos interese, por ejemplo, llevar un contador que digan cuántas instancias se han creado.

En nuestro caso, de todas formas, se trata meramente de constantes por lo que tiene pleno sentido definirlos para toda la clase.

Podemos agregar la posibilidad de sumar y restar tiempos:

In [ ]:
class Tiempo:
    __DIAS = 0
    __HORAS = 1
    __MINUTOS = 2
    __SEGUNDOS = 3
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        segundos = dhms[3]
        minutos = segundos // 60 + dhms[2]
        segundos = segundos % 60
        horas = minutos // 60 + dhms[1]
        minutos = minutos % 60
        dias = horas // 24 + dhms[0]
        horas = horas % 24
        self.__tiempo = [dias, horas, minutos, segundos]
    # getters
    def dhms(self):
        return tuple(self.__tiempo)
    def dias(self):
        return self.__tiempo[Tiempo.__DIAS]
    def horas(self):
        return self.__tiempo[Tiempo.__HORAS]
    def minutos(self):
        return self.__tiempo[Tiempo.__MINUTOS]
    def segundos(self):
        return self.__tiempo[Tiempo.__SEGUNDOS]
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__tiempo[Tiempo.__DIAS] = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__tiempo[Tiempo.__HORAS] = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: minutos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__MINUTOS] = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: segundos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__SEGUNDOS] = segundos
    def __str__(self):
        return str(self.dias()) + 'd' + str(self.horas()) + 'h' + str(self.minutos()) + 'm' + str(self.segundos()) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i == len(self.__tiempo)
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i < len(self.__tiempo) and self.__tiempo[i] < otro.__tiempo[i]
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro
    def __add__(self, otro):
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        res = [0]*4
        for i in range(4):
            res[i] = self.__tiempo[i] + otro.__tiempo[i]
        return Tiempo(tuple(res))

In [ ]:
tiempo = Tiempo((10,13,32,18))
tiempo2 = Tiempo((1,3,42,58))
print(tiempo)
print(tiempo2)
tiempo3 = tiempo + tiempo2
print(tiempo)
print(tiempo2)
print(tiempo3)

10d13h32m18s
1d3h42m58s
10d13h32m18s
1d3h42m58s
11d17h15m16s


Pero si quisiéramos hacer lo mismo para la resta, no funcionaría:

In [ ]:
class Tiempo:
    __DIAS = 0
    __HORAS = 1
    __MINUTOS = 2
    __SEGUNDOS = 3
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        segundos = dhms[3]
        minutos = segundos // 60 + dhms[2]
        segundos = segundos % 60
        horas = minutos // 60 + dhms[1]
        minutos = minutos % 60
        dias = horas // 24 + dhms[0]
        horas = horas % 24
        self.__tiempo = [dias, horas, minutos, segundos]
    # getters
    def dhms(self):
        return tuple(self.__tiempo)
    def dias(self):
        return self.__tiempo[Tiempo.__DIAS]
    def horas(self):
        return self.__tiempo[Tiempo.__HORAS]
    def minutos(self):
        return self.__tiempo[Tiempo.__MINUTOS]
    def segundos(self):
        return self.__tiempo[Tiempo.__SEGUNDOS]
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__tiempo[Tiempo.__DIAS] = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__tiempo[Tiempo.__HORAS] = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: minutos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__MINUTOS] = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: segundos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__SEGUNDOS] = segundos
    def __str__(self):
        return str(self.dias()) + 'd' + str(self.horas()) + 'h' + str(self.minutos()) + 'm' + str(self.segundos()) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i == len(self.__tiempo)
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i < len(self.__tiempo) and self.__tiempo[i] < otro.__tiempo[i]
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro
    def __add__(self, otro):
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        res = [0]*4
        for i in range(4):
            res[i] = self.__tiempo[i] + otro.__tiempo[i]
        return Tiempo(tuple(res))
    def __sub__(self, otro):
        assert isinstance(otro, Tiempo) and otro <= self, 'Error: otro debe ser instancia de Tiempo, y menor que self.'
        res = [0]*4
        for i in range(4):
            res[i] = self.__tiempo[i] - otro.__tiempo[i]
        return Tiempo(tuple(res))


In [ ]:
tiempo = Tiempo((10,13,32,18))
tiempo2 = Tiempo((1,3,42,58))
print(tiempo)
print(tiempo2)
tiempo3 = tiempo + tiempo2
print(tiempo)
print(tiempo2)
print(tiempo3)
tiempo4 = tiempo3 - tiempo2
print(ftiempo4)

Lo resolvemos con un método oculto para "normalizar" o "simplificar" el tiempo (reduciendo los casos en que los segundos o minutos son mayores que 59, o las horas mayores que 23):

In [ ]:
class Tiempo:
    __DIAS = 0
    __HORAS = 1
    __MINUTOS = 2
    __SEGUNDOS = 3
    def __init__(self, dhms : tuple = (0, 0, 0, 0)):
        assert type(dhms) == tuple and all(type(x) == int and x >= 0 for x in dhms), 'Error: intento de crear un tiempo no válido.'
        lista = list(dhms)
        Tiempo.__normalizar_lista(lista)
        self.__tiempo = lista
    def __normalizar_lista(lista):
        segundos = lista[Tiempo.__SEGUNDOS]
        minutos = segundos // 60 + lista[Tiempo.__MINUTOS]
        lista[Tiempo.__SEGUNDOS] = segundos % 60
        horas = minutos // 60 + lista[Tiempo.__HORAS]
        lista[Tiempo.__MINUTOS] = minutos % 60
        lista[Tiempo.__DIAS] = horas // 24 + lista[Tiempo.__DIAS]
        lista[Tiempo.__HORAS] = horas % 24
    # getters
    def dhms(self):
        return tuple(self.__tiempo)
    def dias(self):
        return self.__tiempo[Tiempo.__DIAS]
    def horas(self):
        return self.__tiempo[Tiempo.__HORAS]
    def minutos(self):
        return self.__tiempo[Tiempo.__MINUTOS]
    def segundos(self):
        return self.__tiempo[Tiempo.__SEGUNDOS]
    # setters
    def set_dias(self, dias: int):
        # pre: 0 <= dias
        assert type(dias) == int and dias >= 0,'Error: dias debe ser un número entero no negativo'
        self.__tiempo[Tiempo.__DIAS] = dias
    def set_horas(self, horas: int):
        # pre: 0 <= horas <= 23
        assert type(horas) == int and 0 <= horas <= 23,'Error: horas debe ser un número entero no negativo menor que 24'
        self.__tiempo[Tiempo.__HORAS] = horas
    def set_minutos(self, minutos: int):
        # pre: 0 <= minutos <= 59
        assert type(minutos) == int and 0 <= minutos <= 59,'Error: minutos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__MINUTOS] = minutos
    def set_segundos(self, segundos: int):
        # pre: 0 <= segundos <= 59
        assert type(segundos) == int and 0 <= segundos <= 59,'Error: segundos debe ser un número entero no negativo menor que 60'
        self.__tiempo[Tiempo.__SEGUNDOS] = segundos
    def __str__(self):
        return str(self.dias()) + 'd' + str(self.horas()) + 'h' + str(self.minutos()) + 'm' + str(self.segundos()) + 's'
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i == len(self.__tiempo)
    def __lt__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        i = 0
        while i < len(self.__tiempo) and self.__tiempo[i] == otro.__tiempo[i]:
            i = i + 1
        return i < len(self.__tiempo) and self.__tiempo[i] < otro.__tiempo[i]
    def __le__(self, otro) -> bool:
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        return self < otro or self == otro
    def __add__(self, otro):
        assert isinstance(otro, Tiempo), 'Error: otro debe ser instancia de Tiempo.'
        res = [0]*4
        for i in range(4):
            res[i] = self.__tiempo[i] + otro.__tiempo[i]
        return Tiempo(tuple(res))
    def __sub__(self, otro):
        assert isinstance(otro, Tiempo) and otro <= self, 'Error: otro debe ser instancia de Tiempo, y menor que self.'
        res = [0]*4
        for i in range(4):
            res[i] = self.__tiempo[i] - otro.__tiempo[i]
        Tiempo.__normalizar_lista(res)
        return Tiempo(tuple(res))

In [ ]:
tiempo = Tiempo((10,13,32,18))
tiempo2 = Tiempo((1,3,42,58))
print(tiempo)
print(tiempo2)
tiempo3 = tiempo + tiempo2
print(tiempo)
print(tiempo2)
print(tiempo3)
tiempo4 = tiempo3 - tiempo2
print(tiempo4)

10d13h32m18s
1d3h42m58s
10d13h32m18s
1d3h42m58s
11d17h15m16s
10d13h32m18s


# Fracciones irreducibles

Otro ejemplo interesante es el de las fracciones irreducibles, que llamamos Racional por su correspondencia con los números racionales:

In [ ]:
class Racional:
    def __init__(self, num = 0, den = 1):
        assert type(num) == type(den) == int and den != 0, 'Error: intento de crear fracción no válida.'
        abs_num = abs(num)
        abs_den = abs(den)
        dcm = Racional.__dcm(abs_num, abs_den) # __dcm es un método de la clase, se lo invoca con Racional.__dcm
        self.__numerador = abs_num // dcm
        self.__denominador = abs_den // dcm
        self.__signo = 1 if num * den >= 0 else -1
    def __dcm(a, b): # método oculto para calcular el divisor común mayor de 2 enteros no negativos, b > 0
        n, m = a, b
        while n > 0 and m > 0:
            if n >= m:
                n = n % m
            else:
                m = m % n
        return max(n, m)
    def __mcm(a, b): # método oculto para calcular el múltiplo común menor de 2 enteros positivos
        return a * b // Racional.__dcm(a, b) # __dcm es un método de la clase, se lo invoca con Racional.__dcm
    # getters
    def numerador(self):
        return self.__signo * self.__numerador
    def denominador(self):
        return self.__denominador
    # setters, no queremos que racional sea mutable, no hay setters
    def __str__(self):
        return str(self.numerador()) + '/' + str(self.denominador())
    def __eq__(self, otro) -> bool:
        assert isinstance(otro, Racional), 'Error: otro debe ser instancia de Racional.'
        return self.numerador() == otro.numerador() and self.denominador() == otro.denominador()
    def __add__(self, otro):
        assert isinstance(otro, Racional), 'Error: otro debe ser instancia de Racional.'
        denominador_comun = Racional.__mcm(self.denominador(), otro.denominador()) # __mcm es un método de la clase, se lo invoca con Racional.__mcm
        numerador = self.numerador() * (denominador_comun // self.denominador()) + otro.numerador() * (denominador_comun // otro.denominador())
        return Racional(numerador, denominador_comun)
    def __mul__(self, otro):
        assert isinstance(otro, Racional), 'Error: otro debe ser instancia de Racional.'
        numerador = self.numerador() * otro.numerador()
        denominador = self.denominador() * otro.denominador()
        return Racional(numerador, denominador)

In [ ]:
r1 = Racional()
r2 = Racional(1,2)
r3 = Racional(-2,-4)
print(r1)
print(r2)
print(r3)
r4 = r2 + r3
print(r1)
print(r2)
print(r3)
print(r4)
r5 = r2 * Racional(2,-3)
print(r5)

0/1
1/2
1/2
0/1
1/2
1/2
1/1
-1/3


Queda como ejercicio definir otros métodos para las fracciones irreducibles: resta, división, opuesto, inverso, etc.